In [ ]:
import os
import requests
import time
from datetime import datetime, timedelta, time as dtime
import pytz
import pandas as pd
from tabulate import tabulate
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

PUSHOVER_API_TOKEN = os.getenv('PUSHOVER_API_TOKEN')
PUSHOVER_USER_KEY = os.getenv('PUSHOVER_USER_KEY')
TENANT_ID = os.getenv('TENANT_ID')
TIMEZONE = os.getenv('TIMEZONE', 'Europe/Warsaw')

# Resource mapping from resource_id to court names.
resource_mapping = {
    "81863e50-95cd-4781-82c0-f074047ecd2f": "Kort_1",
    "fe346b27-20d2-4a06-b8cc-ea1b6807c033": "Kort_2",
    "659f26e7-1409-40e5-b19d-f0cc05709223": "Kort_3",
    "16956c5f-c8ac-409e-b461-14a13c4fe226": "Kort_4",
    "57dd9762-57ac-4746-8c0f-1ec7306cb1cb": "Kort_5",
    "6adbec60-7e35-4e8e-a9f4-b49810597547": "Kort_6",
    "d7e15103-a57e-462a-9dbd-4c71c4e3f79c": "Kort_7",
    "8fd738e8-341b-43d2-b5cb-e923f8acc60c": "Kort_8",
    "846e12b4-8e94-4d3d-8273-5a00b0dc0eec": "Kort_9"
}

def get_warsaw_dates(weekend=True):
    warsaw_tz = pytz.timezone(TIMEZONE)
    current_time = datetime.now(warsaw_tz)
    
    date_list = []
    for i in range(15):
        current_date = current_time + timedelta(days=i)
        if weekend or current_date.weekday() not in [5, 6]:
            date_list.append(current_date.strftime('%Y-%m-%d'))
    print("Checking availability of the courts for the next two weeks")
    print("------------------------------------------------------")
    return date_list

def send_pushover_notification(message, user_key, api_token):
    url = 'https://api.pushover.net/1/messages.json'
    data = {
        'token': api_token,
        'user': user_key,
        'message': message,
        'title': 'Court Availability Notification'
    }
    response = requests.post(url, data=data)
    if response.status_code == 200:
        print("Notification sent successfully.")
    else:
        print(f"Failed to send notification: {response.status_code} - {response.text}")

def fetch_available_slots(
    tenant_id: str,
    date_str: str,
    desired_start_min: dtime,
    desired_start_max: dtime,
    desired_duration: int,
    user_key: str,
    api_token: str,
    send_notification: bool
):
    results = []
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    day_name = date_obj.strftime('%A').lower()
    start_min = date_obj.strftime('%Y-%m-%dT00:00:00')
    start_max = (date_obj + timedelta(days=1)).strftime('%Y-%m-%dT00:00:00')

    url = 'https://api.playtomic.io/v1/availability'
    params = {
        'sport_id': 'PADEL',
        'start_min': start_min,
        'start_max': start_max,
        'tenant_id': tenant_id
    }

    headers = {
        'Content-Type': 'application/json',
        'User-Agent': 'iOS 18.3.1',
        'X-Requested-With': 'com.playtomic.app 6.13.0'
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        blocked_ids = {
            "648a9a72-cc0e-4061-8f45-8fd42fe70338",
            "dfc644f3-88ea-439f-9e2b-cbef817f59f8"
        }
        data = [res for res in data if res.get('resource_id') not in blocked_ids]

        utc_tz = pytz.utc
        local_tz = pytz.timezone(TIMEZONE)
        found_slots = False

        for resource in data:
            resource_id = resource.get('resource_id')
            court_name = resource_mapping.get(resource_id, resource_id)
            start_date = resource.get('start_date')
            slots = resource.get('slots', [])

            for slot in slots:
                combined_datetime_str = f"{start_date} {slot.get('start_time')}"
                naive_datetime = datetime.strptime(combined_datetime_str, '%Y-%m-%d %H:%M:%S')
                utc_datetime = utc_tz.localize(naive_datetime)
                local_datetime = utc_datetime.astimezone(local_tz)
                start_local_time = local_datetime.time()
                duration = slot.get('duration')

                if (desired_start_min <= start_local_time <= desired_start_max) and (duration == desired_duration):
                    price = slot.get('price')
                    message_text = f"Court: {court_name}, Start Time: {local_datetime.strftime('%Y-%m-%d %H:%M:%S')}, Duration: {duration} min, Price: {price}"
                    results.append({
                        "date": date_str,
                        "court": court_name,
                        "start_time": local_datetime.strftime('%Y-%m-%d %H:%M:%S'),
                        "day": day_name,
                        "duration": duration,
                        "price": price,
                        "note": "Open slot"
                    })
                    if send_notification:
                        send_pushover_notification(message_text, user_key, api_token)
                    found_slots = True

        if not found_slots:
            results.append({
                "date": date_str,
                "court": "",
                "start_time": "",
                "day": day_name,
                "duration": "",
                "price": "",
                "note": "No available slots"
            })
    else:
        print(f"Failed to retrieve data for {date_str}: {response.status_code} - {response.text}")

    return results

def run_scan(tenant_id, desired_start_min, desired_start_max, desired_duration,
             pushover_user_key, pushover_api_token, send_notification, date_list):
    all_slots = []
    for date_str in date_list:
        slots = fetch_available_slots(
            tenant_id=tenant_id,
            date_str=date_str,
            desired_start_min=desired_start_min,
            desired_start_max=desired_start_max,
            desired_duration=desired_duration,
            user_key=pushover_user_key,
            api_token=pushover_api_token,
            send_notification=send_notification
        )
        all_slots.extend(slots)
    return all_slots

def filter_open_slots(slots):
    open_slots = set()
    for slot in slots:
        if slot["note"] == "Open slot":
            open_slots.add((
                slot["date"],
                slot["court"],
                slot["start_time"],
                slot["day"],
                slot["duration"],
                slot["price"]
            ))
    return open_slots

if __name__ == "__main__":
    tenant_id = TENANT_ID
    desired_start_min = dtime(17, 0)
    desired_start_max = dtime(22, 0)
    desired_duration = 90
    pushover_user_key = PUSHOVER_USER_KEY
    pushover_api_token = PUSHOVER_API_TOKEN
    send_notification = False
    weekend = False

    date_list = get_warsaw_dates(weekend=weekend)

    print("\nInitial scan...")

    initial_slots = run_scan(tenant_id, desired_start_min, desired_start_max,
                             desired_duration, pushover_user_key, pushover_api_token,
                             send_notification, date_list)
    initial_open_slots = filter_open_slots(initial_slots)

    df_initial = pd.DataFrame(initial_slots)
    df_initial = df_initial[["date", "court", "start_time", "day", "duration", "price", "note"]]
    print(tabulate(df_initial, headers='keys', tablefmt='psql', showindex=False))

    print("\nStarting periodic scans every 2 minutes...\n")
    start_time = time.time()

    while True:
        elapsed_seconds = int(time.time() - start_time)
        elapsed_str = str(timedelta(seconds=elapsed_seconds))
        print(f"Running time: {elapsed_str}", end='\r', flush=True)

        time.sleep(120)

        current_slots = run_scan(tenant_id, desired_start_min, desired_start_max,
                                 desired_duration, pushover_user_key, pushover_api_token,
                                 send_notification, date_list)
        current_open_slots = filter_open_slots(current_slots)

        new_open_slots = current_open_slots - initial_open_slots

        if new_open_slots:
            print("\nNew available slots detected:")
            new_slots_list = []
            for slot in new_open_slots:
                new_slots_list.append({
                    "date": slot[0],
                    "court": slot[1],
                    "start_time": slot[2],
                    "day": slot[3],
                    "duration": slot[4],
                    "price": slot[5],
                    "note": "Open slot"
                })
            df_new = pd.DataFrame(new_slots_list)
            df_new = df_new[["date", "court", "start_time", "day", "duration", "price", "note"]]
            print(tabulate(df_new, headers='keys', tablefmt='psql', showindex=False))
            initial_open_slots = initial_open_slots.union(new_open_slots)
            df_initial = pd.concat([df_initial, df_new], ignore_index=True)


Checking availability of the courts for the next two weeks
------------------------------------------------------

Initial scan...
+------------+---------+---------------------+-----------+------------+---------+--------------------+
| date       | court   | start_time          | day       | duration   | price   | note               |
|------------+---------+---------------------+-----------+------------+---------+--------------------|
| 2025-05-05 |         |                     | monday    |            |         | No available slots |
| 2025-05-06 |         |                     | tuesday   |            |         | No available slots |
| 2025-05-07 |         |                     | wednesday |            |         | No available slots |
| 2025-05-08 |         |                     | thursday  |            |         | No available slots |
| 2025-05-09 | Kort_5  | 2025-05-09 22:00:00 | friday    | 90         | 232 PLN | Open slot          |
| 2025-05-12 | Kort_6  | 2025-05-12 22:00:00 